In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pyodbc
import urllib
import joblib
import json
from sqlalchemy import create_engine 
from datetime import datetime, date
import sys
pd.set_option('display.max_columns', None)

### Get Closest Date for each IBVStatusID

In [6]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER

drop table if EXISTS #t1
select ibvT.IBVStatusID, A.Application_ID, A.PortfolioID, A.CustomerID, A.ApplicationDate, CustEmail,
case when ApplicationSteps like '%S%' then 1 else 0 end as NewlyScored,
case when A.ApplicationStatus in ('A','P') then 1 else 0 end as Accepted,
case when A.ApplicationStatus in ('A','P') then A.LeadPurchasePrice else 0 end as LeadPurchasePrice,
case when L.LoanStatus not in ('V','W','G','K') and A.ApplicationStatus='J'  then 1 else 0 end as PartialPreApproved,
case when L.LoanStatus not in ('V','W','G','K')  then 1 else 0 end as Originated,
L.LoanID, datediff(year, VW.DOB, A.ApplicationDate) as Age,
case when VW.Frequency in ('B','S') then 'B' else VW.Frequency end as Frequency,
case when L.LoanStatus not in ('V','W','G','K') then L.OriginatedAmount else null end as OriginatedAmount,
LP.Provider_name, OriginationDate, LOANSTATUS,
case when ((L.RenewalLoanId <> '0') or (A.LPCampaign = 'RENEW'))  then 'RENEWAL' when ApplicationSteps not like '%R%' then 'NEW' else 'REPEAT' end as CustType, RenewalLoanId
into #t1
from Application A
left join Loans L on A.Application_ID = L.ApplicationID and A.PortFolioID = L.PortFolioID
left join LeadProvider LP on A.LeadProviderID = LP.LeadProviderID
left join LF_LMS_Logs..VW_ApplicationDump VW on A.APPGUID = VW.APPGUID
left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.AccountNumber = A.CustomerSSN
--left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.ApplicationID = A.Application_ID AND ibvT.PortfolioID= A.PortfolioID
where A.ApplicationDate>'2023-04-30' and ((CustEmail is null) or  (CustEmail not in ( 'josh@concordadvice.com', 'tiffany.speedyservicing@gmail.com','bobby@speedyservicing.com',
'and.kom@example.com', 'and.tor020@example.com', 'and.tor050@example.com', 'and.torrc4@example.com',
'test@dot818.com', 'test@example.com', 'test@gmail.com', 'test@loanscanada.ca', 'test@test.com', 'test2@test.com',
'testerqad@gmail.com', 'testik@test.com', 'testteam@dmaassociatescorp.com', 'tiffany.speedyservicing@gmail.com')))

UPDATE #t1 -- update the renewal loanid from 0 to correct previous loanIDs
SET #t1.RenewalLoanId = B.RenewalLoanId
from #t1 A inner join (select * from
(select A.LoanID, L.LoanID as RenewalLoanId, row_number() over (partition by A.LoanID order by datediff(day, L.OriginationDate, A.OriginationDate) desc) as RN from #t1 A
inner join Application A2 on A.CustomerID = A2.CustomerID and A2.ApplicationDate < A.ApplicationDate
inner join Loans L on A2.Application_ID = L.ApplicationID and A2.PortFolioID = L.PortFolioID and L.OriginationDate < A.OriginationDate and L.LoanStatus not in ('W','V','G','K')
where A.CustType = 'RENEWAL' and A.RenewalLoanId = '0') K where RN=1) B on A.LoanID = B.LoanID



"""
query2 = """
SELECT *
FROM #t1
"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf = df_perf.rename(columns = {'Application_ID':'AppID'})
df_perf.AppID = df_perf.AppID.astype(int)


In [7]:
# peak the performance table
print(df_perf.shape)
print(df_perf.head(3))

(196661, 20)
   IBVStatusID  AppID  PortfolioID  CustomerID         ApplicationDate  \
0       4372.0   9896            1        4521 2023-11-27 14:16:51.673   
1       4372.0   9897            1        4521 2023-11-27 14:16:53.557   
2       4374.0   8420            1        4522 2023-10-26 10:38:21.177   

                       CustEmail  NewlyScored  Accepted  LeadPurchasePrice  \
0              corrieg@gmail.com            0         0                0.0   
1              corrieg@gmail.com            0         0                0.0   
2  christopher.mambo99@gmail.com            0         0                0.0   

   PartialPreApproved  Originated LoanID   Age Frequency  OriginatedAmount  \
0                   0           0   None  63.0         B               NaN   
1                   0           0   None  63.0         B               NaN   
2                   0           0   None  21.0         B               NaN   

  Provider_name OriginationDate LOANSTATUS CustType RenewalLoanI

#### Join with the IBVStautsID Dedup first and then rerun the bankuity to see if there are improvements

In [3]:
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

cnxn = engine.connect()

query1 = f""" 
SELECT * 
FROM [LF_LMS_Logs].[dbo].[CModelLogs] 
WHERE modelName = 'ibv'

"""

df_ibvIDs = pd.read_sql(query1, cnxn)

In [4]:
print(df_ibvIDs.head(3))
print(df_ibvIDs.shape)

   CModelLogID ModelName  ApplicationID  PortfolioID  IBVStatusID  \
0         1794       ibv              0           -1         4244   
1         2560       ibv              0           -1         3158   
2         2624       ibv              0           -1         4125   

            DateProcessed                                       ModelRequest  \
0 2025-05-23 11:25:17.140  {"asOfDate":"2025-05-23","accounts":[{"account...   
1 2025-06-17 10:13:20.697  {"asOfDate":"2025-06-17","accounts":[{"account...   
2 2025-06-18 06:54:28.840  {"asOfDate":"2025-06-18","accounts":[{"account...   

                                       ModelResponse  ProcessingTime  
0  {"accounts":[{"accountGuid":"wPoOvmamKYUgELV6k...            2577  
1  {"accounts":[{"accountGuid":"pQrj3v1nKriQLQkgw...            2969  
2  {"accounts":[{"accountGuid":"6gg76em73vIwN3aOD...            3625  
(16976, 9)


In [5]:
total_rows = len(df_ibvIDs)
unique_ids = df_ibvIDs['IBVStatusID'].nunique(dropna=True)
duplicate_count = total_rows - unique_ids
print(f"Total rows: {total_rows}")
print(f"Unique IBVStatusID: {unique_ids}")
print(f"Duplicate count: {duplicate_count}")

Total rows: 16976
Unique IBVStatusID: 10269
Duplicate count: 6707


In [ ]:
# for the duplicated IBVStatusID, first Join the df_ibvIDs with the df_perf, 
# and then use the ApplicationDate to check which one is the closest to the DateProcessed. Keep the one with the closest ApplicationDate.

#df_ibvIDs['IBVStatusID'].dtype == Int64
#df_perf['IBVStatusID'].dtype == float64

In [8]:
# Join with df_perf and keep, per IBVStatusID, the row whose DateProcessed is closest to ApplicationDate
required_perf_cols = {'IBVStatusID', 'ApplicationDate'}
required_ibv_cols = {'IBVStatusID', 'DateProcessed'}

missing_perf = required_perf_cols - set(df_perf.columns)
missing_ibv = required_ibv_cols - set(df_ibvIDs.columns)
assert not missing_perf, f"df_perf missing columns: {missing_perf}"
assert not missing_ibv, f"df_ibvIDs missing columns: {missing_ibv}"

# Ensure datetime types
_df_perf = df_perf.copy()
_df_perf['ApplicationDate'] = pd.to_datetime(_df_perf['ApplicationDate'], errors='coerce')
_df_ibv = df_ibvIDs.copy()
_df_ibv['DateProcessed'] = pd.to_datetime(_df_ibv['DateProcessed'], errors='coerce')
_df_perf['IBVStatusID'] = pd.to_numeric(_df_perf['IBVStatusID'], errors='coerce').astype('Int64')
_df_ibv['IBVStatusID'] = pd.to_numeric(_df_ibv['IBVStatusID'], errors='coerce').astype('Int64')

# Merge keeping all df_perf columns (except key); drop overlapping cols from left to avoid duplicates
merged = _df_ibv.merge(_df_perf, on='IBVStatusID', how='left', suffixes=('', '_perf'))

# Compute absolute time delta (seconds) where possible
merged['abs_diff_seconds'] = (merged['DateProcessed'] - merged['ApplicationDate']).abs().dt.total_seconds()



In [9]:
merged.head()

,CModelLogID,ModelName,ApplicationID,PortfolioID,IBVStatusID,DateProcessed,ModelRequest,ModelResponse,ProcessingTime,AppID,PortfolioID_perf,CustomerID,ApplicationDate,CustEmail,NewlyScored,Accepted,LeadPurchasePrice,PartialPreApproved,Originated,LoanID,Age,Frequency,OriginatedAmount,Provider_name,OriginationDate,LOANSTATUS,CustType,RenewalLoanId,abs_diff_seconds
0,1794,ibv,0,-1,4244,2025-05-23 11:25:17.140,"{""asOfDate"":""2025-05-23"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""wPoOvmamKYUgELV6k...",2577,8221.0,1.0,4427.0,2023-10-23 14:59:31.000,Barbaraalbert8@gmail.com,1.0,1.0,5.0,0.0,1.0,I2846-0,57.0,B,500.0,Dot818,2023-10-24 12:35:13.473,D,NEW,0,49926346.14
1,1794,ibv,0,-1,4244,2025-05-23 11:25:17.140,"{""asOfDate"":""2025-05-23"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""wPoOvmamKYUgELV6k...",2577,13237.0,1.0,4427.0,2024-01-21 07:58:25.000,barbaraalbert8@gmail.com,0.0,1.0,0.0,0.0,1.0,I4010-0,58.0,B,500.0,WEB,2024-01-25 13:49:53.207,D,REPEAT,0,42175612.14
2,1794,ibv,0,-1,4244,2025-05-23 11:25:17.140,"{""asOfDate"":""2025-05-23"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""wPoOvmamKYUgELV6k...",2577,13423.0,1.0,4427.0,2024-01-24 13:36:04.870,barbaraalbert8@gmail.com,0.0,0.0,0.0,0.0,0.0,None,58.0,B,NaN,WEB,NaT,None,REPEAT,None,41896152.27
3,1794,ibv,0,-1,4244,2025-05-23 11:25:17.140,"{""asOfDate"":""2025-05-23"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""wPoOvmamKYUgELV6k...",2577,13424.0,1.0,4427.0,2024-01-24 13:37:05.860,barbaraalbert8@gmail.com,0.0,0.0,0.0,0.0,0.0,None,58.0,B,NaN,WEB,NaT,None,REPEAT,None,41896091.28
4,1794,ibv,0,-1,4244,2025-05-23 11:25:17.140,"{""asOfDate"":""2025-05-23"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""wPoOvmamKYUgELV6k...",2577,18511.0,1.0,4427.0,2024-04-16 19:34:02.000,None,0.0,1.0,0.0,0.0,1.0,I5765-0,NaN,None,700.0,WEB,2024-04-16 19:41:40.670,D,RENEWAL,I4010-0,34703475.14


In [10]:
# For groups with valid ApplicationDate + DateProcessed, pick min delta; else pick most recent DateProcessed
selected_idx = []
for ibv_id, grp in merged.groupby('IBVStatusID', sort=False):
    grp_valid = grp.copy()
    if grp_valid['ApplicationDate'].notna().any() and grp_valid['DateProcessed'].notna().any():
        sel = grp_valid['abs_diff_seconds'].fillna(float('inf')).idxmin()
    elif grp_valid['DateProcessed'].notna().any():
        sel = grp_valid['DateProcessed'].idxmax()
    else:
        sel = grp_valid.index[0]
    selected_idx.append(sel)

# Deduped frame: one row per IBVStatusID
df_ibvIDs_closest = merged.loc[selected_idx].sort_values('IBVStatusID').reset_index(drop=True)

# Report
total_rows = len(df_ibvIDs)
unique_ids = df_ibvIDs['IBVStatusID'].nunique(dropna=True)
print(f"Input rows: {total_rows} | Unique IBVStatusID: {unique_ids} | Kept: {len(df_ibvIDs_closest)}")
matched = df_ibvIDs_closest['ApplicationDate'].notna().sum()
print(f"Rows with ApplicationDate available for matching: {matched}/{len(df_ibvIDs_closest)}")

# CSV-ready subset: IBVStatusID + DateCreated (using the chosen DateProcessed)
df_ibvIDs_for_csv = df_ibvIDs_closest[['IBVStatusID', 'DateProcessed']].rename(columns={'DateProcessed': 'DateCreated'})

df_ibvIDs_for_csv.head(10)

Input rows: 16976 | Unique IBVStatusID: 10269 | Kept: 10269
Rows with ApplicationDate available for matching: 10144/10269


,IBVStatusID,DateCreated
0,757,2025-12-04 07:43:09.047
1,1258,2025-12-04 14:02:28.853
2,1382,2025-12-04 11:43:58.047
3,1810,2025-12-04 07:53:09.990
4,2334,2025-12-04 10:18:45.287
5,2632,2025-12-04 10:45:14.617
6,3158,2025-06-17 10:13:20.697
7,3509,2025-12-04 07:57:36.090
8,3513,2025-12-04 07:30:27.137
9,3586,2025-12-04 09:01:27.960


In [11]:
df_ibvIDs_closest.head()
print(df_ibvIDs_closest.shape)

(10269, 29)


In [12]:
df_ibvIDs_closest[df_ibvIDs_closest['IBVStatusID'] == 66030]

,CModelLogID,ModelName,ApplicationID,PortfolioID,IBVStatusID,DateProcessed,ModelRequest,ModelResponse,ProcessingTime,AppID,PortfolioID_perf,CustomerID,ApplicationDate,CustEmail,NewlyScored,Accepted,LeadPurchasePrice,PartialPreApproved,Originated,LoanID,Age,Frequency,OriginatedAmount,Provider_name,OriginationDate,LOANSTATUS,CustType,RenewalLoanId,abs_diff_seconds
4793,13829,ibv,0,-1,66030,2025-12-04 14:02:06.730,"{""asOfDate"":""2025-08-26"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""e997f18e-9ab5-437...",2500,107390.0,1.0,58503.0,2025-08-26 08:11:53,lohnesadam25@gmail.com,1.0,1.0,12.0,0.0,1.0,I28374-0,36.0,B,600.0,Loans Canada,2025-08-26 15:20:03.713,R,NEW,0,8661013.73


#### Export the IBVStatusIDs + DateCreated for the Bankuity Rerun

In [ ]:
""" import os

out_csv = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_ibv_shadowV5_dedup.csv"
os.makedirs(os.path.dirname(out_csv), exist_ok=True)

df_ibvIDs_for_csv.to_csv(out_csv, index=False)
out_csv
 """

'/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_ibv_shadowV5_dedup.csv'

### Azure Job Submission Output Log Analysis

In [14]:
# Combined: parse failed IDs, join with df_ibvIDs_closest, robust coverage, and missing preview
import re
import pandas as pd

log_path = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_data/1124_log.txt"

# Parse unique failed IBVStatusIDs from log
failed_ids = []
with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        m = re.search(r"IBVStatusID\s+(\d+)\s+failed", line, flags=re.IGNORECASE)
        if m:
            failed_ids.append(int(m.group(1)))

df_failed = pd.DataSource(data={"IBVStatusID": sorted(set(failed_ids))}) if False else pd.DataFrame(sorted(set(failed_ids)), columns=["IBVStatusID"])  # keep DataFrame
print(f"Failed count (unique): {len(df_failed)}")

# Join to bring ModelRequest/DateProcessed from df_ibvIDs_closest
assert 'IBVStatusID' in df_ibvIDs_closest.columns, "df_ibvIDs_closest missing 'IBVStatusID'"
df_failed_details = df_failed.merge(df_ibvIDs_closest, on='IBVStatusID', how='left')

# Naive presence vs robust non-empty check
present_naive = int(df_failed_details['ModelRequest'].notna().sum()) if 'ModelRequest' in df_failed_details.columns else 0
mr_str = df_failed_details['ModelRequest'].astype(str) if 'ModelRequest' in df_failed_details.columns else pd.Series([], dtype=str)
has_model_request = (
    df_failed_details['ModelRequest'].notna()
    & mr_str.str.strip().ne("")
    & mr_str.str.strip().str.lower().ne("nan")
) if 'ModelRequest' in df_failed_details.columns else pd.Series([], dtype=bool)

present_robust = int(has_model_request.sum()) if len(has_model_request) else 0
print(f"Joined rows: {len(df_failed_details)} | with ModelRequest (naive): {present_naive} | non-empty: {present_robust} | empty/NA: {len(df_failed_details)-present_robust}")



Failed count (unique): 3451
Joined rows: 3451 | with ModelRequest (naive): 3451 | non-empty: 154 | empty/NA: 3297


#### Creating Seperate dataframes for both empty and non-empty model requests

In [19]:
# Split failed IDs into non-empty and empty ModelRequest sets
assert 'df_failed_details' in globals() and 'has_model_request' in globals(), "Run the combined join/coverage cell first"

subset_non_empty = (
    df_failed_details.loc[has_model_request, ['IBVStatusID']]
    .drop_duplicates()
    .sort_values('IBVStatusID')
    .reset_index(drop=True)
)
subset_empty = (
    df_failed_details.loc[~has_model_request, ['IBVStatusID']]
    .drop_duplicates()
    .sort_values('IBVStatusID')
    .reset_index(drop=True)
)

print(f"Non-empty ModelRequest IDs: {len(subset_non_empty)}")
print(f"Empty ModelRequest IDs: {len(subset_empty)}")

subset_non_empty.head(10), subset_empty.head(10)


Non-empty ModelRequest IDs: 154
Empty ModelRequest IDs: 3297


(   IBVStatusID
 0         4125
 1        11277
 2        11364
 3        12170
 4        12198
 5        12653
 6        13962
 7        13998
 8        14730
 9        15931,
    IBVStatusID
 0         7997
 1        10558
 2        16992
 3        23026
 4        26423
 5        27341
 6        29074
 7        30289
 8        30817
 9        37195)

In [35]:
# print the two dataframes to rerun_output_data
df_ibvIDs_closest.to_csv("/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_data/df_ibvIDs_perf.csv", index=False)


#### Write IBVStatusIDs with non-empty model requests

In [ ]:
import os, json
output_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_JSONs"
os.makedirs(output_dir, exist_ok=True)

written = 0
parse_errors = []
for ibv_id, model_request in zip(subset_empty['IBVStatusID'], subset_empty['ModelRequest']):
    try:
        if isinstance(model_request, (dict, list)):
            data = model_request
        else:
            s = str(model_request).strip()
            data = json.loads(s)
        out_path = os.path.join(output_dir, f"failedModelRequest_{int(ibv_id)}.json")
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, separators=(',', ': '), ensure_ascii=False)
        written += 1
    except Exception as e:
        parse_errors.append((int(ibv_id), str(e)))

print(f"Wrote {written} JSON files to {output_dir}")
if parse_errors:
    print(f"Parse/Write errors: {len(parse_errors)} (first 5 shown)")
    print(parse_errors[:5])

#### Write IBVStatusIDs with empty model requests

In [22]:
# write IBVStatusIDs with empty model requests
# Write placeholders for IDs with empty ModelRequest
import os, json

assert 'subset_empty' in globals(), "Run the split step to build subset_empty first"

written = 0
for ibv_id in subset_empty['IBVStatusID']:
    out_path = os.path.join(output_dir, f"emptyIDs_{int(ibv_id)}.json")
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump({"IBVStatusID": int(ibv_id), "hasModelRequest": False}, f, indent=2)
    written += 1

print(f"Wrote {written} empty ID placeholders to {output_dir}")



Wrote 3297 empty ID placeholders to /Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_JSONs
